In [1]:
import numpy as np
import yaml
import cv2
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [5]:
#Path Config
path = "dataset-1.0/"
subpath_annotations = "annotations/"
subpath_images = "images/"

In [6]:
#Read Annotations Filenames
annotation_path = lambda i: path+subpath_annotations+"%03d"%(i)+"_annotation.yaml"
annotations = [annotation_path(i+1) for i in range(60)]
print(annotations)

['dataset-1.0/annotations/001_annotation.yaml', 'dataset-1.0/annotations/002_annotation.yaml', 'dataset-1.0/annotations/003_annotation.yaml', 'dataset-1.0/annotations/004_annotation.yaml', 'dataset-1.0/annotations/005_annotation.yaml', 'dataset-1.0/annotations/006_annotation.yaml', 'dataset-1.0/annotations/007_annotation.yaml', 'dataset-1.0/annotations/008_annotation.yaml', 'dataset-1.0/annotations/009_annotation.yaml', 'dataset-1.0/annotations/010_annotation.yaml', 'dataset-1.0/annotations/011_annotation.yaml', 'dataset-1.0/annotations/012_annotation.yaml', 'dataset-1.0/annotations/013_annotation.yaml', 'dataset-1.0/annotations/014_annotation.yaml', 'dataset-1.0/annotations/015_annotation.yaml', 'dataset-1.0/annotations/016_annotation.yaml', 'dataset-1.0/annotations/017_annotation.yaml', 'dataset-1.0/annotations/018_annotation.yaml', 'dataset-1.0/annotations/019_annotation.yaml', 'dataset-1.0/annotations/020_annotation.yaml', 'dataset-1.0/annotations/021_annotation.yaml', 'dataset-1.0

In [8]:
#Read annotation contents
annotations_json = []
for annotation in annotations:
    with open(annotation, 'r') as stream:
        try:
            annotations_json.append(yaml.load(stream))
            print(annotations_json)
        except yaml.YAMLError as exc:
            print(exc)

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

...

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

[{'annotation': [{'points': {'x': [810.0, 841.0, 846.0, 926.0, 956.0, 1054.0, 1175.0, 1161.0, 1010.0, 971.0, 948.0, 937.0, 909.0, 837.0], 'y': [225.0, 234.0, 266.0, 338.0, 408.0, 422.0, 317.0, 230.0, 126.0, 114.0, 130.0, 167.0, 170.0, 169.0]}, 'type': 'weed'}, {'points': {'x': [1070.0, 1055.0, 980.0, 850.0, 834.0, 767.0, 758.0, 772.0, 756.0, 682.0, 637.0, 646.0, 681.0, 663.0, 721.0, 770.0, 851.0, 888.0, 939.0, 987.0], 'y': [626.0, 722.0, 739.0, 658.0, 748.0, 730.0, 674.0, 654.0, 624.0, 635.0, 570.0, 558.0, 523.0, 496.0, 490.0, 476.0, 517.0, 537.0, 533.0, 545.0]}, 'type': 'weed'}, {'points': {'x': [1153.0, 1102.0, 1095.0, 1149.0, 1177.0], 'y': [882.0, 872.0, 926.0, 948.0, 933.0]}, 'type': 'crop'}, {'points': {'x': [925.0, 787.0, 727.0, 708.0, 743.0, 742.0, 849.0, 898.0, 918.0, 919.0], 'y': [769.0, 750.0, 736.0, 750.0, 790.0, 856.0, 918.0, 918.0, 871.0, 797.0]}, 'type': 'weed'}, {'points': {'x': [706.0, 737.0, 741.0, 693.0, 604.0, 552.0, 550.0, 578.0, 649.0, 669.0, 708.0, 745.0, 762.0, 7

In [13]:
for annotation in annotations_json:
    # filename path
    image_file = annotation['filename']
    print(image_file)
    # load a color image
    print((path+subpath_images+image_file))
    # this is a list of regions described by a serie of points
    regions = annotation['annotation']
    for region in regions:
        points = region['points']       
        px = points['x']
        py = points['y']
        if(type(px) is list):
            pts = np.array([[int(x),int(y)] for x,y in zip(px,py)], dtype=np.int32)
            print(pts)

001_image.png
dataset-1.0/images/001_image.png
[[ 810  225]
 [ 841  234]
 [ 846  266]
 [ 926  338]
 [ 956  408]
 [1054  422]
 [1175  317]
 [1161  230]
 [1010  126]
 [ 971  114]
 [ 948  130]
 [ 937  167]
 [ 909  170]
 [ 837  169]]
[[1070  626]
 [1055  722]
 [ 980  739]
 [ 850  658]
 [ 834  748]
 [ 767  730]
 [ 758  674]
 [ 772  654]
 [ 756  624]
 [ 682  635]
 [ 637  570]
 [ 646  558]
 [ 681  523]
 [ 663  496]
 [ 721  490]
 [ 770  476]
 [ 851  517]
 [ 888  537]
 [ 939  533]
 [ 987  545]]
[[1153  882]
 [1102  872]
 [1095  926]
 [1149  948]
 [1177  933]]
[[925 769]
 [787 750]
 [727 736]
 [708 750]
 [743 790]
 [742 856]
 [849 918]
 [898 918]
 [918 871]
 [919 797]]
[[706 754]
 [737 788]
 [741 858]
 [693 877]
 [604 799]
 [552 715]
 [550 680]
 [578 665]
 [649 670]
 [669 653]
 [708 639]
 [745 632]
 [762 648]
 [749 680]]
[[505 604]
 [479 569]
 [442 591]
 [438 650]
 [441 696]
 [459 724]
 [444 731]
 [409 734]
 [386 735]
 [359 764]
 [370 781]
 [424 779]
 [471 773]
 [497 763]
 [512 739]
 [510 695]
 

 [622 368]]
013_image.png
dataset-1.0/images/013_image.png
[[686 370]
 [674 310]
 [548 346]
 [554 459]
 [609 473]
 [703 474]
 [707 424]]
[[532 459]
 [531 373]
 [388 453]
 [375 496]
 [383 525]
 [458 526]
 [512 473]]
[[270 426]
 [175 280]
 [102 351]
 [ 84 451]
 [155 558]
 [253 567]]
[[726 627]
 [696 591]
 [652 588]
 [628 603]
 [625 630]
 [662 647]
 [700 647]
 [727 638]]
[[792 157]
 [738 202]
 [706 307]
 [706 323]
 [713 336]
 [735 337]
 [749 314]
 [801 308]
 [831 307]
 [845 294]
 [842 250]
 [811 165]]
[[672 750]
 [690 749]
 [709 770]
 [729 784]
 [769 777]
 [784 781]
 [773 845]
 [749 912]
 [710 954]
 [683 963]
 [675 933]]
[[788 641]
 [751 664]
 [736 695]
 [706 698]
 [688 721]
 [685 742]
 [692 750]
 [719 777]
 [731 782]
 [759 777]
 [777 777]
 [789 781]
 [815 772]
 [833 726]
 [827 694]]
[[831 537]
 [834 493]
 [771 487]
 [731 496]
 [703 497]
 [695 516]
 [706 582]
 [709 602]
 [724 623]
 [729 628]
 [726 638]
 [715 655]
 [708 690]
 [718 697]
 [728 695]
 [746 651]
 [788 630]
 [816 616]
 [823 578]